In [37]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Hyperparameters


In [38]:
COLUMNS_TO_KEEP = [
    "direct_rad:W",
    "clear_sky_rad:W",
    "diffuse_rad:W",
    "direct_rad_1h:J",
    "is_in_shadow:idx",
    "clear_sky_energy_1h:J",
    "diffuse_rad_1h:J",
    "is_day:idx",
    "sun_elevation:d",
    # "ceiling_height_agl:m",
    # "effective_cloud_cover:p",
    # "visibility:m",
    # "date_calc",
    "pv_measurement",
]
LEARNING_RATE = 0.001
NUM_EPOCHS = 100
BATCH_SIZE = 32
NUM_FEATURES = len(COLUMNS_TO_KEEP) - 1  # -1 because pv_measurement is the target
FEATURE_SIZE = 4  # 7 days of hourly data
WEIGHT_DECAY = 0.01
SEQUENCE_LENGTH = 7*24

# Neural net


In [39]:
def create_sequences(data, sequence_length):
    """
    Converts time series data into overlapping sequences/windows.
    """
    sequences = []
    target_length = 1
    for i in range(len(data) - sequence_length - target_length + 1):
        seq = data[i : i + sequence_length]
        sequences.append(seq)
    return np.array(sequences)

class SolarPredictionNet(nn.Module):
    def __init__(self, sequence_length, num_channels):
        super(SolarPredictionNet, self).__init__()

        self.conv1 = nn.Conv1d(num_channels, 32, kernel_size=sequence_length, stride=1)
        self.conv2 = nn.Conv1d(32, 64, kernel_size=1, stride=1)
        self.fc1 = nn.Linear(64, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = x.view(x.size(0), -1)  # Flatten
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        return self.fc3(x)

# Load dataset


In [40]:
# Load data from Parquet files
df_data = pd.read_parquet("data/A/X_train_observed.parquet")
df_target = pd.read_parquet("data/A/train_targets.parquet")

# Merge the datasets
df_merged = pd.merge(
    df_data, df_target, left_on="date_forecast", right_on="time", how="inner"
)
df_merged = df_merged[COLUMNS_TO_KEEP]

y = df_merged["pv_measurement"]
X = df_merged.drop("pv_measurement", axis=1)

# Convert dataframes to sequences
X_sequences = create_sequences(X.values, SEQUENCE_LENGTH)
# Adjust the sequence creation for y
y_sequences = y.values[SEQUENCE_LENGTH-1:-1]  # Aligned with the end of each sequence and remove the last element

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(
    X_sequences, y_sequences, test_size=0.2, random_state=42
)

# Normalize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
X_val = scaler.transform(X_val.reshape(-1, X_val.shape[-1])).reshape(X_val.shape)

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).transpose(1, 2)  # Adjust shape to [batch, channels, sequence]
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32).transpose(1, 2)  # Adjust shape to [batch, channels, sequence]
y_val_tensor = torch.tensor(y_val, dtype=torch.float32).unsqueeze(1)


# Create a custom dataset
class SolarDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]


# Create datasets for training and validation
train_dataset = SolarDataset(X_train_tensor, y_train_tensor)
val_dataset = SolarDataset(X_val_tensor, y_val_tensor)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

# ... [Neural Network and Training code from previous messages]


# Training Loop


In [41]:
def train_model(model, train_loader, val_loader):
    criterion = nn.L1Loss()
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)

    for epoch in range(NUM_EPOCHS):
        model.train()
        for batch_idx, (data, target) in enumerate(train_loader):
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()

        # Evaluate the model on the validation set
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for data, target in val_loader:
                output = model(data)
                loss = criterion(output, target)
                val_loss += loss.item()

        # Average validation loss
        val_loss /= len(val_loader)

        print(
            f"Epoch {epoch + 1}/{NUM_EPOCHS}, Training Loss: {loss.item()}, Validation Loss: {val_loss}"
        )

    print("Training complete!")

In [42]:
model = SolarPredictionNet(SEQUENCE_LENGTH, NUM_FEATURES)
train_model(model, train_loader, val_loader)

Epoch 1/100, Training Loss: 182.9816131591797, Validation Loss: 255.39696378965635
Epoch 2/100, Training Loss: 96.17887115478516, Validation Loss: 247.17413925995697
Epoch 3/100, Training Loss: 88.5373764038086, Validation Loss: 231.7605849394927
Epoch 4/100, Training Loss: 77.52349853515625, Validation Loss: 253.63495355554528
Epoch 5/100, Training Loss: 40.405303955078125, Validation Loss: 225.09522533932247
Epoch 6/100, Training Loss: 54.99589538574219, Validation Loss: 224.61707289412215
Epoch 7/100, Training Loss: 78.37740325927734, Validation Loss: 231.013224050161
Epoch 8/100, Training Loss: 46.74898147583008, Validation Loss: 228.03345011118296
Epoch 9/100, Training Loss: 46.962799072265625, Validation Loss: 219.42545821731156
Epoch 10/100, Training Loss: 74.39583587646484, Validation Loss: 223.9060064161146
Epoch 11/100, Training Loss: 54.16368103027344, Validation Loss: 217.37908404582257
Epoch 12/100, Training Loss: 36.5133171081543, Validation Loss: 217.42653062150285
Epoch